In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, sum, mean
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as F

In [3]:
# Initialize Spark session
spark = SparkSession.builder.appName("CovidDataAnalysis").getOrCreate()

# Read the CSV file into a DataFrame
df = spark.read.csv('./complete.csv', header=True, inferSchema=True)


In [4]:
# Convert state names to lowercase
df = df.withColumn('Name of State / UT', lower(col('Name of State / UT')))

# Show the 'Name of State / UT' column
df.select('Name of State / UT').show(truncate=False)


+------------------+
|Name of State / UT|
+------------------+
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
|kerala            |
+------------------+
only showing top 20 rows



In [5]:
# Group by date and sum the new cases
max_cases_day = df.orderBy(col("New cases").desc()).select("Date", "New cases").first()
print(f"The day with the greatest number of COVID cases is: {max_cases_day['Date']} with {max_cases_day['New cases']} cases.")

The day with the greatest number of COVID cases is: 2020-07-26 with 18366 cases.


In [6]:
# Group by state and sum the total confirmed cases
state_cases = df.groupBy('Name of State / UT').agg(sum('Total Confirmed cases').alias('Total Confirmed Cases'))

# Get the state with the second-largest number of cases
state_cases = state_cases.orderBy(col('Total Confirmed Cases').desc())
second_largest_state_row = state_cases.collect()[1]  # Index 1 for the second-largest
second_largest_state = second_largest_state_row['Name of State / UT']
second_largest_cases = second_largest_state_row['Total Confirmed Cases']

print(f"The state with the second-largest number of COVID cases is {second_largest_state} with {second_largest_cases} cases.")


The state with the second-largest number of COVID cases is tamil nadu with 7847083.0 cases.


In [7]:
# Filter for Union Territories and find the one with the least number of deaths
ut_df = df.filter(col("Name of State / UT").contains("union territory"))

# Group by Union Territory and sum the deaths
ut_deaths = ut_df.groupBy("Name of State / UT").agg(sum("Death").alias("Total Deaths"))

# Find the Union Territory with the least number of deaths
ut_least_deaths_row = ut_deaths.orderBy(col("Total Deaths")).first()
ut_least_deaths = ut_least_deaths_row['Name of State / UT']
least_deaths = ut_least_deaths_row['Total Deaths']

print(f"The Union Territory with the least number of deaths is: {ut_least_deaths} with {least_deaths} deaths.")



The Union Territory with the least number of deaths is: union territory of ladakh with 0.0 deaths.


In [8]:
# Calculate the Death to Total Confirmed cases ratio and find the state with the lowest ratio
df = df.withColumn("DeathToConfirmedRatio", col("Death") / col("Total Confirmed cases"))
lowest_ratio_state = df.orderBy(col("DeathToConfirmedRatio").asc()).select("Name of State / UT", "DeathToConfirmedRatio").first()
print(f"The state with the lowest Death to Total Confirmed cases ratio is: {lowest_ratio_state['Name of State / UT']} with a ratio of {lowest_ratio_state['DeathToConfirmedRatio']}.")


The state with the lowest Death to Total Confirmed cases ratio is: puducherry with a ratio of None.


In [9]:
# Convert Date column to date format and extract month
df = df.withColumn('Date', F.to_date(col('Date'), 'yyyy-MM-dd'))
df = df.withColumn('Month', F.date_format(col('Date'), 'yyyy-MM'))

# Group by month and sum the new recovered cases
monthly_recovered = df.groupBy('Month').agg(sum('New recovered').alias('Total New Recovered'))

max_recovered_month_row = monthly_recovered.orderBy(col('Total New Recovered').desc()).first()
max_recovered_month = max_recovered_month_row['Month']
max_recovered = max_recovered_month_row['Total New Recovered']

print(f"The month with the most new recovered cases was {max_recovered_month} with {max_recovered} recoveries.")


The month with the most new recovered cases was 2020-07 with 722983 recoveries.
